# Filters performance logs out of an iOS log dump
Log file aquired by recording with Console.app, CMD-A, CMD-C in the log list and CMD-V in a new text file inside /raw

(!) filter logs in Console.app by searching for: `Process:PlatformChannelPerformance category:measure`





In [13]:
from datetime import date

# datetime object containing current date and time
today = date.today()

experiment = "scroll"
trace_keyword = "PlatformChannelPerformance	" # for XCode logs
#trace_keyword = "PlatformChannelPerformance	" # for Console.app logs
#trace_keyword = "screenUpdate" #for XCode logs of scroll tests
input_file_name = "raw/2023-11-30-scroll.txt"
output_file_name = "csv/{}_{}.csv".format(today, experiment)
headers = "name,time"

In [14]:
import re

# open the input file for reading
with open(input_file_name, 'r') as input_file:
    # read in all lines from the input file
    lines = input_file.readlines()

# check if the line corresponds to a log output (either has the prefix included or is just a line without timestamp)
def is_log(input_string):
    return (trace_keyword in input_string) or ((not input_string[0].isdigit()) and ("Test Suite"  not in input_string) and ("Test Case"  not in input_string))

# cut off everything before measure and the process id:, including the word itself
def remove_prefix(input_string):
    index = input_string.find(trace_keyword)
    if index != -1:
        return input_string[(index + len(trace_keyword)):]
    return input_string

# https://stackoverflow.com/a/59082116
def replace_last_occurrence(input_string):
    return ','.join(input_string.rsplit(':', 1))

# filter input to just contain lines with the keyword
filtered_lines = [line for line in lines if is_log(line)]

# cut off everything before measure:, including the word itself
formatted_lines = [remove_prefix(line) for line in filtered_lines]

csv_lines = [replace_last_occurrence(line) for line in formatted_lines]

# open the output file for writing
with open(output_file_name, 'w') as output_file:
    # write out the filtered lines to the output file
    output_file.write(headers + "\n")
    output_file.writelines(csv_lines)
